In [14]:
import numpy as np
import math


#code and test voltage controle class on 28/11/2020

In [15]:
class BasicElectrical:
    
    def __init__(self,accuracy=3,freq=50):

        self.accuracy = accuracy

        self.freq = freq
    
    def power(self,V,I):

        return V*(I.conjugate())
    
    def PhaseDiff(self,a,b):
        
        alpha = math.atan(a.imag/a.real)
        
        beta = math.atan(b.imag/b.real)
        
        return (alpha - beta)
    
    def polar(self,r,theta,accuracy=3):
        
        theta = theta*np.pi/180
        
        x = round(r*np.cos(theta),accuracy)
        
        y = round(r*np.sin(theta),accuracy)
        
        return complex(x,y)
        
    def ABCD_T(self,Z,Y):
        
        A = 1+(Y*Z)/2 ; B= Z*(1+(Y*Z)/4) ; C = Y ; D = A
        
        return np.asarray([[A,B],[C,D]])
    
    def Source_T(self,Vr,Ir,Z,Y):
        
        R = np.asarray([[Vr],[Ir]])
        
        A = self.ABCD_T(Z,Y)
        
        return np.matmul(A*R)
    
    def ABCD_PI(self,Z,Y):
        
        A = 1+(Y*Z)/2 ; B=Z ; C=Y(1+(Y*z)/4); D = A
        
        return np.asarray([[A,B],[C,D]])
    
    def Source_PI(self,Vr,Ir,Z,Y):
        
        R = np.asarray([[Vr],[Ir]])
        
        A = self.ABCD_PI(Z,Y)
        
        return np.matmul(A*R)
    
    def ABCD_Long(self,z,y,length):
        
        gama = pow(z*y,0.5)
        
        Zc = pow(z/y,0.5)
        
        A = np.cosh(gama*length) ; B = Zc*np.sinh(gama*length) ; C = np.sinh(gama*length)/Zc ; D = A
        
        return np.asarray([[A,B],[C,D]])
    
    def Source_Long(self,Vr,Ir,z,y,length):
        
        R = np.asarray([[Vr],[Ir]])
        
        A = self.ABCD_Long(z,y,length)
        
        return np.matmul(A*R)
    
    def setprecision(self,*args):
    
        A = map(lambda x:complex(round(x.real,self.accuracy),round(x.imag,self.accuracy)),args)

        return list(A)
    
    def PhaseComponents(self,Va0,Va1,Va2):
        
        Lambda = self.polar(1,120)

        Va = Va0+Va1+Va2

        Vb = Va0+Lambda*Lambda*Va1+Lambda*Va2

        Vc = Va0+Lambda*Va1+Lambda*Lambda*Va2

        V = [Va,Vb,Vc]

        V = map(lambda x:complex(round(x.real,3),round(x.imag,3)),V)

        return V
    
    def SymetricComponents(self,Va,Vb,Vc):
        
        Lambda = self.polar(1,120)

        Va0 = (1/3)*(Va+Vb+Vc)

        Va1 = (1/3)*(Va+Lambda*Vb+Lambda*Lambda*Vc)

        Va2 = (1/3)*(Va+Lambda*Lambda*Vb+Lambda*Vc)

        V = [Va0,Va1,Va2]

        V = map(lambda x:complex(round(x.real,3),round(x.imag,3)),V)

        return list(V)

In [16]:
class VoltageControle(BasicElectrical):
    
    def __init__(accuracy=3):

        super().__init__(accuracy=accuracy)

    def SynchronousMachine(self,E,V,X,delta):
        
        delta = np.pi/180
        
        P = abs(E)*abs(V)*np.sin(delta)/X
   
        Q = (abs(E)*abs(V)*np.cos(delta)/X) - (pow(abs(V),2)/X)
    
        return complex(P,Q)
    
    def TapChaningTransformer(self,V1,V2,P,Q,R,X):
        
        ans = (V2/V1)*(1/(1-(R*P+X*Q)/(V1*V2)))
        
        return pow(ans,0.5)
    
    def CircleParams(self,Vs,Vr,A,B,flag=False):
        
        beta = np.arctan(B.Imag/B.real)/np.pi
        
        alpha = np.arctan(A.Imag/A.real)/np.pi
        
        D = abs(A)*pow(abs(Vs),2)/abs(B)
        
        x = D*np.cos(beta-alpha)
        
        y = D*np.sin(beta-alpha)
        
        r = pow(abs(Vs)*abs(Vr)/abs(B),2)
        
        if flag is False:
        
            print(f'Center (x,y) => {x,y} radius => {r}')
            
            print(f'Distance B/w origin and center {D} at an angle {180*(beta-alpha)}')
            
            return -x,-y,r
            
        else:
            
            print(f'Center (x,y) => {-x,-y} radius => {r}')
            
            print(f'Distance B/w origin and center {D} at an angle {180*(beta-alpha)}')
            
            return x,y,r
    
    def SendingEndCircle(self,Vs,Vr,D,B):
        
        return self.CircleParams(Vs,Vr,D,B,flag=False)
    
    def ReceiveEndCircle(self,Vs,Vr,A,B):
        
        return self.CircleParams(Vs,Vr,A,B,flag=True)
        
        
        

In [31]:
class Faults(BasicElectrical):
    
    def __init__(self,Ea,z0=None,z1=None,z2=None,zn=None,zf=None,accuracy=3):
        
        super().__init__(accuracy=accuracy)
        
        self.Ea = Ea
        
        self.z0 = z0
        
        self.z1 = z1
        
        self.z2 = z2
        
        self.zn = zn
        
        self.zf = zf
        
    def FaultData(self,Va0,Va1,Va2,Ia0,Ia1,Ia2):
        
        [Va0,Va1,Va2] = self.setprecision(Va0,Va1,Va2)

        [Ia0,Ia1,Ia2] = self.setprecision(Ia0,Ia1,Ia2)

        [Va,Vb,Vc] = self.PhaseComponents(Va0,Va1,Va2)

        [Ia,Ib,Ic] = self.PhaseComponents(Ia0,Ia1,Ia2)

        data = {'Ia0':Ia0,'Ia1':Ia1,'Ia2':Ia2,
                'Va0':Va0,'Va1':Va1,'Va2':Va2,
                'Va':Va,'Vb':Vb,'Vc':Vc,
                'Ia':Ia,'Ib':Ib,'Ic':Ic}
        
        return data   
    
    def SingleLineFault(self):

        Ia1 = (self.Ea)/(self.z0+self.z1+self.z2)

        Ia0 = Ia1

        Ia2 = Ia1

        Va1 = self.Ea - Ia1*self.z1

        Va2 = -Ia2*self.z2

        Va0 = -Ia0*self.z0

        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        for key,data in FaultData.items():
            print(key+":",data)

        return FaultData

    def LineToLineFault(self):

        Ia0 = 0

        Ia1 = (self.Ea)/(self.z1+self.z2)

        Ia2 = -Ia1

        Va1 = self.Ea - Ia1*self.z1

        Va2 = -Ia2*self.z2

        Va0 = -Ia0*self.z0

        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        for key,data in FaultData.items():
            print(key+":",data)


        return FaultData

    def DoubleLineGroundFault(self):

        Ia1 = (self.Ea)/(self.z1+(self.z0*self.z2)/(self.z0+self.z2))

        Va1 = self.Ea - Ia1*self.z1

        Va0 = Va1

        Va2 = Va1

        Ia0 = -Va0/self.z0

        Ia2 = -Va2/self.z2
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        for key,data in FaultData.items():
            print(key+":",data)

        return FaultData

    def ThreePhaseFault(self):

        Ia0 = 0

        Ia1 = self.Ea/self.z1

        Ia2 = 0

        Va0 = 0

        Va1 = 0

        Va2 = 0

        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        for key,data in FaultData.items():
            print(key+":",data)

        return FaultData
    
    def SingleLineFaultZ(self):
    
        Ia1 = (self.Ea)/(self.z0+self.z1+self.z2+3*(self.zn+self.zf))
        
        Ia2 = Ia1
        
        Ia0 = Ia1
        
        Va0 = -Ia0*(self.z0+3*self.zn)
        
        Va1 = Ea - Ia1*self.z1
        
        Va2 = -Ia2*self.z2
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        for key,data in FaultData.items():
            print(key+":",data)
        
        return FaultData

    def LineToLineFaultZ(self):

        Ia1 = (self.Ea)/(self.z1+self.z2+self.zf)
        
        Ia2 = -Ia1
        
        Ia0 = 0
        
        Va1 = Ea - Ia1*self.z1
        
        Va2 = -Ia2*self.z2
        
        Va0 = -Ia0*(self.z0)
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        for key,data in FaultData.items():
            print(key+":",data)
        
        return FaultData

    def DoubleLineFaultz(self):

        zeq = self.z2*(self.z0+3*self.zn+3*self.zf)/(self.z2+self.z0+3*self.zn+3*self.zf)

        Ia1 = (self.Ea)/(self.z1+zeq)
        
        Ia2 = -1*(self.Ea-self.Ia1*self.z1)/(self.z2)
        
        Ia0 = -1*(self.Ea-Ia1*self.z1)/(self.z0+3*(self.zn+self.zf))
        
        Va1 = Ea - Ia1*self.z1
        
        Va2 = -Ia2*self.z2
        
        Va0 = -Ia0*(self.z0+3*self.zn)
        
        FaultData = self.FaultData(Va0,Va1,Va2,Ia0,Ia1,Ia2)
        
        for key,data in FaultData.items():
            print(key+":",data)

        return FaultData

In [32]:
SymetricComponents = BasicElectrical().SymetricComponents

Va = complex(500,150)

Vb = complex(100,-600)

Vc = complex(-300,600)

print(SymetricComponents(Va,Vb,Vc))

[(100+50j), (546.396+165.475j), (-146.399-65.475j)]


In [33]:
SymetricComponents = BasicElectrical().SymetricComponents

polar = BasicElectrical().polar

Va = polar(100,0)

Vb = polar(33,-100)

Vc = polar(38,176.5)

print(SymetricComponents(Va,Vb,Vc))

[(18.78-10.06j), (50.66+14.325j), (30.559-4.265j)]


In [37]:
z0 = complex(0,0.1)

z1 = complex(0,0.25)

z2 = complex(0,0.35)

Ea = 1

faults = Faults(Ea,z0,z1,z2)

FaultData = faults.DoubleLineGroundFault()

Ia0: (-0+2.373j)
Ia1: -3.051j
Ia2: (-0+0.678j)
Va0: (0.237+0j)
Va1: (0.237+0j)
Va2: (0.237+0j)
Va: (0.711+0j)
Vb: 0j
Vc: 0j
Ia: 0j
Ib: (-3.229+3.559j)
Ic: (3.229+3.56j)


In [25]:
E = BasicElectrical()

z = complex(10,35.1)

y = complex(0,3.13*pow(10,-4))

A = E.ABCD_T(z,y)

In [26]:
A

array([[0.99450685+1.56500000e-03j, 9.9450685 +3.50114202e+01j],
       [0.        +3.13000000e-04j, 0.99450685+1.56500000e-03j]])

In [27]:
R = np.asarray([[66],[E.polar(218.96,-36.86)]])

In [28]:
R

array([[ 66.     +0.j   ],
       [175.191-131.346j]])

In [29]:
S = np.matmul(A,R)

In [30]:
S

array([[6406.53394757+4827.54404212j],
       [ 174.43420605 -130.32966481j]])